In [31]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

import numpy as np
import os

np.random.seed(42)
tf.random.set_seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deploy"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [32]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis].astype(np.float32) / 255.
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

import googleapiclient.discovery
#2017250045 정태환
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "double-balm-289912-929a9a390dff.json"

project_id="double-balm-289912"
model_id = "my_mnist_model"
model_path = "projects/{}/models/{}".format(project_id, model_id)
ml_resource = googleapiclient.discovery.build("ml", "v1").projects()


In [38]:
def predict(X):
    input_data_json = {"signature_name": "serving_default",
                       "instances": X.tolist()}
    request = ml_resource.predict(name=model_path, body=input_data_json)
    response = request.execute()
    if "error" in response:
        raise RuntimeError(response["error"])
    return np.array([pred['dense_5'] for pred in response["predictions"]])
#2017250045 정태환
X_new = X_test[:10]
print("\npredict(X_test[:10]):", predict(X_new))


predict(X_test[:10]): [[1.14266695e-04 1.51848894e-07 9.78322350e-04 2.77053635e-03
  3.75338755e-06 7.67125384e-05 3.91618222e-08 9.95572805e-01
  5.34381252e-05 4.29933396e-04]
 [8.13938328e-04 3.53349169e-05 9.88318563e-01 7.00815767e-03
  1.27453163e-07 2.32063452e-04 2.56750453e-03 9.68847114e-10
  1.02423911e-03 8.75037642e-08]
 [4.43073295e-05 9.70304430e-01 9.04920883e-03 2.26388546e-03
  4.86470060e-04 2.87215714e-03 2.27167783e-03 8.36741924e-03
  4.04331507e-03 2.97109393e-04]
 [9.99276578e-01 1.37288518e-08 1.24055747e-04 1.22131341e-05
  2.03612316e-07 5.33860002e-05 4.80883376e-04 1.42408226e-05
  1.52733937e-05 2.31991453e-05]
 [4.62775759e-04 4.79039591e-06 2.46720063e-03 1.54007081e-04
  9.61716294e-01 2.70371529e-04 1.67213683e-03 1.68396556e-03
  5.38360968e-04 3.10300011e-02]
 [4.83169924e-06 9.92127299e-01 9.45044565e-04 7.30966043e-04
  4.30491891e-05 1.69096777e-04 4.95482745e-05 4.37504333e-03
  1.43692654e-03 1.18245647e-04]
 [2.66675470e-05 1.81869802e-06 2.7